In [ ]:
import duckdb
import pygwalker
import pandas as pd
import lonboard
from ipywidgets import widgets, IntRangeSlider, jsdlink

from load import DATABASE_PATH

pd.set_option("display.max_columns", 50)

## Explore source data

In [ ]:
with duckdb.connect(str(DATABASE_PATH)) as connection:
    tables = connection.sql("SHOW ALL TABLES").df()
tables

Multiple instances of `pygwalker` may slow down the Jupyter notebook

In [ ]:
# conn = pygwalker.data_parsers.database_parser.Connector(
#     f"duckdb:///{DATABASE_PATH}", "SELECT * FROM farmers_markets"
# )
# pygwalker.walk(conn, kernel_computation=True)

In [ ]:
# conn = pygwalker.data_parsers.database_parser.Connector(
#     f"duckdb:///{DATABASE_PATH}", "SELECT * FROM garden_block"
# )
# pygwalker.walk(conn, kernel_computation=True)

In [ ]:
# conn = pygwalker.data_parsers.database_parser.Connector(
#     f"duckdb:///{DATABASE_PATH}", "SELECT * FROM garden_info"
# )
# pygwalker.walk(conn, kernel_computation=True)

In [ ]:
# conn = pygwalker.data_parsers.database_parser.Connector(
#     f"duckdb:///{DATABASE_PATH}", "SELECT * FROM garden_info"
# )
# pygwalker.walk(conn, kernel_computation=True)

In [ ]:
# common_values_parksid = len(set(garden_info.parksid).intersection(garden_block_lot.parksid))
# print(
#   f"""
#     Unique values in garden_info:{len(garden_info.parksid.drop_duplicates())}
#     Unique values in garden_block_lot:{len(garden_block_lot.parksid.drop_duplicates())}
#     Comon values:{common_values_parksid}
#   """
# )

In [ ]:
with duckdb.connect(str(DATABASE_PATH)) as connection:
    farms_description = connection.sql("DESCRIBE farms").df()
farms_description

In [ ]:
with duckdb.connect(str(DATABASE_PATH)) as connection:
    farms = connection.sql("SELECT DISTINCT * FROM farms").df()
farms

In [ ]:
farms.borough.value_counts(dropna=False)

In [ ]:
with duckdb.connect(str(DATABASE_PATH)) as connection:
    markets_description = connection.sql("DESCRIBE markets").df()
markets_description

In [ ]:
with duckdb.connect(str(DATABASE_PATH)) as connection:
    markets = connection.sql("SELECT DISTINCT * FROM markets").df()
markets

In [ ]:
markets.borough.value_counts(dropna=False)

## Try transforming data

In [ ]:
far_garden = "RGT001"
mid_garden = "R146-GT001"
near_garden = "R109-GT001"
weird_garden = "RGT009"
si_market = "Heritage Farm Stand (Gazebo Road)"

In [ ]:
special_farms = farms[farms["parksid"].isin([near_garden, mid_garden, far_garden, weird_garden])]
special_farms

In [ ]:
special_markets = markets[markets["name"].isin([si_market])]
special_markets

In [ ]:
with duckdb.connect(str(DATABASE_PATH)) as connection:
    duckdb.load_extension("spatial", connection=connection)
    layer_gardens = lonboard.ScatterplotLayer.from_duckdb(
        "SELECT parksid, gardenname, point_geometry_wgs84 FROM farms",
        con=connection,
        radius_min_pixels=5,
        radius_max_pixels=10,
        get_fill_color=[100, 150, 100],
    )
    layer_markets = lonboard.ScatterplotLayer.from_duckdb(
        "SELECT name, point_geometry_wgs84 FROM markets",
        con=connection,
        radius_min_pixels=5,
        radius_max_pixels=10,
        get_fill_color=[100, 100, 100],
    )

In [ ]:
with duckdb.connect(str(DATABASE_PATH)) as connection:
    farm_to_market_distance_ft = connection.sql(
        "SELECT distance_ft FROM farm_to_market"
    ).df()["distance_ft"]
max_filter_range = max(farm_to_market_distance_ft) / 10

In [ ]:
extension = lonboard.layer_extension.DataFilterExtension()

with duckdb.connect(str(DATABASE_PATH)) as connection:
    duckdb.load_extension("spatial", connection=connection)
    layer_garden_to_markets = lonboard.PathLayer.from_duckdb(
        "SELECT gardenname, market_name, distance_ft, line_geometry_wgs84 FROM farm_to_market",
        con=connection,
        crs="EPSG:4326",
        get_color=[150, 120, 120],
        opacity=0.3,
        width_min_pixels=2,
        extensions=[extension],
        get_filter_value=farm_to_market_distance_ft,
        filter_range=[0, max_filter_range],
    )

In [ ]:
slider = IntRangeSlider(
    value=(0, 10000),
    min=0,
    max=max_filter_range,
    step=1,
    description="farm_to_market_distance_ft: ",
    style={"description_width": "initial"},
)

jsdlink((slider, "value"), (layer_garden_to_markets, "filter_range"))
slider.layout = widgets.Layout(width="800px")

In [ ]:
m = lonboard.Map(
    [
        layer_garden_to_markets,
        layer_gardens,
        layer_markets,
    ]
)
m

In [ ]:
slider